# load data

In [1454]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics import mean_squared_error


In [1461]:
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train_y=train['playtime_forever']
train_X=train.drop(['playtime_forever'],axis=1)
X_tr,X_te , y_tr, y_te = train_test_split(train_X, train_y, random_state=41, test_size=0.1)

# resample

In [1456]:
resample_data=X_tr
resample_data['playtime_forever']=y_tr

c:\users\irisg\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1457]:
#negative class 
# class which is no class_0=seperate_data(class_0)need for regresion
class_0= resample_data[resample_data['playtime_forever']==0]
class_1= resample_data[(resample_data['playtime_forever']>0) & (resample_data['playtime_forever']<3)]
class_2= resample_data[(resample_data['playtime_forever']>3) & (resample_data['playtime_forever']<10)]
class_3= resample_data[(resample_data['playtime_forever']>10) & (resample_data['playtime_forever']<30)]
class_4= resample_data[(resample_data['playtime_forever']>30) & (resample_data['playtime_forever']<70)]
class_5= resample_data[resample_data['playtime_forever']>70]

In [1458]:
after_resample_data=pd.concat([class_0,
                                class_1,
                                class_2,class_2,class_2,class_2,
                                class_3,class_3,class_3,class_3,class_3,class_3,class_3,class_3,class_3,class_3,
                                class_4,class_4,class_4,class_4,class_4,class_4,class_4,class_4,class_4,class_4,
                                class_5,class_5,class_5,class_5,class_5,class_5,class_5,class_5,class_5,class_5
                               ])

In [1459]:
after_resample_data=after_resample_data.reset_index(drop=True)
y_tr_resample=after_resample_data['playtime_forever']
X_tr_resample=after_resample_data.drop(['playtime_forever'],axis=1)

# lasso

In [1465]:
from sklearn.linear_model import Lasso
clf=Lasso(alpha=0.1)
clf.fit(X_tr_resample,y_tr_resample)
preds=clf.predict(X_te)
print(mean_squared_error(preds,y_te))
output=clf.predict(test)
results=pd.DataFrame(output,columns=['playtime_forever'])
results['playtime_forever']=results['playtime_forever'].apply(lambda x: 0 if x<0 else x )
results.to_csv('results_lasso.csv',index=True,header=True)
results=pd.read_csv('results_lasso.csv')
results.rename(columns={'Unnamed: 0':'id'})
results.to_csv('results_lasso.csv',index=False,header=True)                    

135.5856854043844


# xgboost

In [1328]:
import xgboost as xgb

In [1329]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [1471]:
bst=xgb.XGBRegressor(objective ='reg:squarederror',
                         learning_rate=0.1,
                          n_estimators=800,
                          max_depth=6,
                          min_child_weight=8,
                          subsample=0.3,
                          colsample_bytree=0.6,
                          reg_alpha=0.3,
                          reg_lambda=1,
                          
                         criterion='mse')
bst.fit(X_tr_resample,y_tr_resample,eval_set=[(X_tr,y_tr),(X_te, y_te)],verbose=False,eval_metric='rmse')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, criterion='mse', gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=8, missing=None, n_estimators=800,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.3, verbosity=1)

In [1472]:
preds=bst.predict(X_te)
print(mean_squared_error(y_te,preds))

22.06570431756743


In [1475]:
output=bst.predict(test)
results=pd.DataFrame(output,columns=['playtime_forever'])
results['playtime_forever']=results['playtime_forever'].apply(lambda x: 0 if x<0 else x )
print(list(results['playtime_forever']))
results.to_csv('results_xgb.csv',index=True,header=True)
results=pd.read_csv('results_xgb.csv')
results.rename(columns={'Unnamed: 0':'id'})
results.to_csv('results_xgb.csv',index=False,header=True)                    

[0.9949638843536377, 6.2404465675354, 0.0, 0.0, 14.515026092529297, 0.780192494392395, 3.8857715129852295, 0.0, 1.1469517946243286, 0.0, 1.1953524351119995, 0.0, 3.7923343181610107, 0.8174365758895874, 8.650744438171387, 0.0, 7.451643466949463, 0.0, 1.0387784242630005, 3.245060443878174, 0.0, 5.331924915313721, 0.0, 4.352076053619385, 0.0, 4.142984390258789, 0.0, 4.311939239501953, 0.9434305429458618, 2.9758858680725098, 2.540360450744629, 2.1216397285461426, 0.0, 0.6456385850906372, 6.411214351654053, 0.0, 1.7106058597564697, 1.8349910974502563, 0.0, 0.7676631808280945, 0.0, 5.08602237701416, 1.2598721981048584, 1.1047914028167725, 2.0036017894744873, 0.0, 0.0, 0.0, 0.0, 3.402886152267456, 4.526651382446289, 3.098552942276001, 0.0, 1.1653926372528076, 1.6156338453292847, 2.3017358779907227, 2.5456855297088623, 2.017796039581299, 0.0, 4.5977559089660645, 5.776065826416016, 2.400259017944336, 0.0, 16.40421485900879, 0.0, 6.7127766609191895, 0.5705493092536926, 1.1880005598068237, 7.4212